In [1]:
import os
import pickle
import sys

import ase
import ase.data
import ase.io
import ase.visualize
import jax
import jax.numpy as jnp
import jraph
import ml_collections
import tqdm
import yaml

sys.path.append('..')

In [8]:
import datatypes
import input_pipeline
import qm9
from analyses import analysis
from models import ATOMIC_NUMBERS, RADII, create_model

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


ModuleNotFoundError: No module named 'utility_classes'

In [3]:
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/mace/interactions=3/l=2/channels=32"
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/e3schnet/interactions=4/l=3/channels=32"
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/mace/interactions=4/l=3/channels=32"
path = "/Users/songk/atomicarchitects/spherical_harmonic_net/workdirs/v3/mace/interactions=4/l=5/channels=32"

with open(path + "/checkpoints/params.pkl", 'rb') as f:
    params = pickle.load(f)
with open(path + "/config.yml", "rt") as config_file:
    config = yaml.unsafe_load(config_file)

assert config is not None
config = ml_collections.ConfigDict(config)

In [4]:
name = path.split("/")[-5:]
name = "_".join(name)
print(name)

v3_mace_interactions=4_l=5_channels=32


In [5]:
model = create_model(config, run_in_evaluation_mode=True)
apply_fn = jax.jit(model.apply)

def apply(frag, seed):
    frags = jraph.pad_with_graphs(frag, 32, 1024, 2)
    preds = apply_fn(params, seed, frags)
    pred = jraph.unpad_with_graphs(preds)
    return pred

In [6]:
def append_pred_to_ase_atoms(molecule: ase.Atoms, pred: datatypes.Predictions) -> ase.Atoms:
    focus = pred.globals.focus_indices.squeeze(0)
    pos_focus = molecule.positions[focus]
    pos_rel = pred.globals.position_vectors.squeeze(0)

    new_specie = jnp.array(
        ATOMIC_NUMBERS[pred.globals.target_species.squeeze(0).item()]
    )
    new_position = pos_focus + pos_rel

    return ase.Atoms(
        positions=jnp.concatenate([molecule.positions, new_position[None, :]], axis=0),
        numbers=jnp.concatenate([molecule.numbers, new_specie[None]], axis=0),
    )


In [7]:
molecules = []

for seed in tqdm.tqdm(range(64)):
    molecule = ase.Atoms(
        positions=jnp.array([[0, 0, 0.0]]),
        numbers=jnp.array([6]),
    )

    rng = jax.random.PRNGKey(seed)
    for step in range(31):
        k, rng = jax.random.split(rng)
        frag = input_pipeline.ase_atoms_to_jraph_graph(
            molecule, ATOMIC_NUMBERS, config.nn_cutoff
        )
        pred = apply(frag, k)

        stop = pred.globals.stop.squeeze(0).item()
        if stop:
            break

        molecule = append_pred_to_ase_atoms(molecule, pred)

    if molecule.numbers.shape[0] < 32:
        molecules.append(molecule)


100%|██████████| 64/64 [31:24<00:00, 29.44s/it]  


In [ ]:
def ase_to_mol_dict(generated_frag: datatypes.Fragments, save=True, model_path=None, file_name=None):
    '''from G-SchNet: https://github.com/atomistic-machine-learning/G-SchNet'''

    generated = (
        {}
    )
    for mol in jraph.unbatch(generated_frag):
        l = mol.nodes.species.shape[0]
        if l not in generated:
            generated[l] = {
                "_positions": np.array([mol.nodes.positions]),
                "_atomic_numbers": np.array([list(map(
                    lambda z: models.ATOMIC_NUMBERS[z],
                    mol.nodes.species
                ))]),
            }
        else:
            generated[l]["_positions"] = np.append(
                generated[l]["_positions"],
                np.array([mol.nodes.positions]),
                0,
            )
            generated[l]["_atomic_numbers"] = np.append(
                generated[l]["_atomic_numbers"],
                np.array([list(map(
                    lambda z: models.ATOMIC_NUMBERS[z],
                    mol.nodes.species
                ))]),
                0,
            )

    return generated

In [10]:
if not os.path.exists(f"gen/{name}"):
    os.mkdir(f"gen/{name}")

for seed, molecule in enumerate(molecules):
    ase.io.write(f"gen/{name}/molecule_{seed}.xyz", molecule)

In [11]:
with open(f'gen/{name}/molecules.pkl', 'wb') as f:
    pickle.dump(molecules, f)